# Prophet Model for Spare Part Demand Forecasting

This notebook implements Facebook Prophet for long-term demand forecasting (30-90 days).

## Objectives
1. Load and prepare data for Prophet
2. Train Prophet model with seasonality
3. Generate forecasts
4. Evaluate model performance
5. Cross-validation
6. Save model for deployment

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded successfully!')

## 1. Load and Prepare Data

In [ ]:
# Load daily aggregated demand
df = pd.read_csv('../data/processed/daily_demand.csv', parse_dates=['date'])
print(f'Loaded {len(df)} rows')
df.head()

In [ ]:
# Prepare data for Prophet (requires 'ds' and 'y' columns)
prophet_df = df[['date', 'demand_quantity']].copy()
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print(f'Prophet data shape: {prophet_df.shape}')
print(f'Date range: {prophet_df["ds"].min()} to {prophet_df["ds"].max()}')
prophet_df.head()

In [ ]:
# Visualize the time series
fig = px.line(prophet_df, x='ds', y='y', title='Daily Demand Time Series')
fig.update_layout(xaxis_title='Date', yaxis_title='Demand')
fig.show()

## 2. Train-Test Split

In [ ]:
# Split data: use last 30 days for testing
test_days = 30
train_df = prophet_df[:-test_days]
test_df = prophet_df[-test_days:]

print(f'Training set: {len(train_df)} days ({train_df["ds"].min()} to {train_df["ds"].max()})')
print(f'Test set: {len(test_df)} days ({test_df["ds"].min()} to {test_df["ds"].max()})')

## 3. Train Prophet Model

In [ ]:
# Initialize Prophet model
model = Prophet(
    seasonality_mode='multiplicative',  # Works better for demand data
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,  # Flexibility of trend
    seasonality_prior_scale=10.0
)

# Add country holidays (India)
model.add_country_holidays(country_name='IN')

print('Prophet model initialized with:')
print('- Multiplicative seasonality')
print('- Yearly + Weekly seasonality')
print('- Indian holidays')

In [ ]:
# Fit the model
print('Training Prophet model...')
model.fit(train_df)
print('Model trained successfully!')

## 4. Generate Forecast

In [ ]:
# Create future dataframe for prediction
future = model.make_future_dataframe(periods=test_days + 30, freq='D')  # +30 for future forecast
print(f'Forecast dataframe: {len(future)} days')
future.tail()

In [ ]:
# Generate predictions
forecast = model.predict(future)
print(f'Forecast generated: {len(forecast)} predictions')
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)

In [ ]:
# Interactive forecast plot
fig = plot_plotly(model, forecast)
fig.update_layout(title='Prophet Demand Forecast')
fig.show()

In [ ]:
# Plot components (trend, seasonality)
fig = plot_components_plotly(model, forecast)
fig.show()

## 5. Model Evaluation

In [ ]:
# Get predictions for test period
test_forecast = forecast[forecast['ds'].isin(test_df['ds'])][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
test_forecast = test_forecast.merge(test_df, on='ds')

print('Test Period Predictions vs Actuals:')
test_forecast[['ds', 'y', 'yhat', 'yhat_lower', 'yhat_upper']].head(10)

In [ ]:
# Calculate metrics
y_true = test_forecast['y']
y_pred = test_forecast['yhat']

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print('='*50)
print('PROPHET MODEL EVALUATION METRICS')
print('='*50)
print(f'MAE  (Mean Absolute Error):     {mae:.2f}')
print(f'RMSE (Root Mean Squared Error): {rmse:.2f}')
print(f'MAPE (Mean Absolute % Error):   {mape:.2f}%')
print('='*50)

In [ ]:
# Visualize actual vs predicted
fig = go.Figure()

fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['y'],
                         mode='lines+markers', name='Actual', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat'],
                         mode='lines+markers', name='Predicted', line=dict(color='orange')))

fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_upper'],
                         mode='lines', name='Upper Bound', line=dict(dash='dash', color='lightgray')))

fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_lower'],
                         mode='lines', name='Lower Bound', line=dict(dash='dash', color='lightgray'),
                         fill='tonexty', fillcolor='rgba(128,128,128,0.2)'))

fig.update_layout(title='Prophet: Actual vs Predicted (Test Period)',
                  xaxis_title='Date', yaxis_title='Demand')
fig.show()

## 6. Cross-Validation

In [ ]:
# Perform cross-validation
print('Running cross-validation (this may take a few minutes)...')

cv_results = cross_validation(
    model,
    initial='365 days',   # Initial training period
    period='30 days',     # Spacing between cutoff dates
    horizon='30 days'     # Forecast horizon
)

print(f'Cross-validation complete: {len(cv_results)} predictions')
cv_results.head()

In [ ]:
# Calculate performance metrics from CV
cv_metrics = performance_metrics(cv_results)
cv_metrics

In [ ]:
# Plot CV metrics over horizon
fig = px.line(cv_metrics, x='horizon', y=['mape', 'mae', 'rmse'],
              title='Cross-Validation Metrics by Forecast Horizon')
fig.update_layout(yaxis_title='Error', xaxis_title='Forecast Horizon')
fig.show()

## 7. Save Model

In [ ]:
# Save the trained model
import os
os.makedirs('../models', exist_ok=True)

model_path = '../models/prophet_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f'Model saved to: {model_path}')

In [ ]:
# Save metrics for comparison
metrics = {
    'model': 'Prophet',
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'cv_mape_mean': cv_metrics['mape'].mean()
}

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('../models/prophet_metrics.csv', index=False)
print('Metrics saved!')
metrics_df

## 8. Future Forecast (Next 30 Days)

In [ ]:
# Get the future predictions (beyond test data)
future_forecast = forecast[forecast['ds'] > prophet_df['ds'].max()][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
future_forecast.columns = ['Date', 'Predicted_Demand', 'Lower_Bound', 'Upper_Bound']

print('Next 30 Days Forecast:')
future_forecast

In [ ]:
# Save future forecast
future_forecast.to_csv('../data/processed/prophet_forecast.csv', index=False)
print('Future forecast saved to: ../data/processed/prophet_forecast.csv')

## Summary

| Metric | Value |
|--------|-------|
| Model | Prophet |
| Training Period | ~700 days |
| Test Period | 30 days |
| MAE | See above |
| RMSE | See above |
| MAPE | See above |

**Notes:**
- Prophet captures yearly and weekly seasonality well
- Indian holidays are included
- Best for long-term forecasts (30-90 days)
- Model saved for deployment

In [ ]:
print('Prophet Model Training Complete!')